#Foundations of Computer Science project

Stefano Colombo
matricola 838941

##Import packages and datasets

In [71]:
import pandas as pd
import numpy as np
import re

In [72]:
gps=pd.read_csv('C://Users//stefa//Desktop//Data Science//Primo anno//Primo Semestre//Foundations of Computer Science//Project//googleplaystore.csv')
gps_ur=pd.read_csv('C://Users//stefa//Desktop//Data Science//Primo anno//Primo Semestre//Foundations of Computer Science//Project//googleplaystore_user_reviews.csv')

Ispezione dei dataset: innanzitutto Google Play Store

In [73]:
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [74]:
gps.shape

(10841, 13)

In [75]:
gps.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

per poi passare a Google Play Store User Reviews

In [76]:
gps_ur.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [77]:
gps_ur.shape

(64295, 5)

In [78]:
gps_ur.dtypes

App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object

Procedendo con il progetto mi sono accorto della presenza di un problema che riguarda la riga 10472 (probabilmente non ha il campo "Category" e tutto risulta traslato verso sinistra)

In [79]:
gps.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


Dunque la scelta migliore è rimuovere tale record

In [80]:
gps=gps.drop(10472)

In [81]:
gps.shape

(10840, 13)

##1. Convert the app sizes to a number

Al fine di aggirare i problemi che potrebbero essere creati da 'Varies with device' mi è sembrato opportuno convertirlo in modo tale da poterlo successivamente riportarlo alla sua forma originale 

In [82]:
gps['Size'] = [re.sub('Varies with device', '99999999', size) for size in gps['Size']] 
gps['Size'] = [re.sub(',', '.', size) for size in gps['Size']]

In [83]:
sizes_to_num_re = re.compile('(?P<amount>\d*\.*\d*)(?P<unit>\w*\+*)') #

def unit_to_mult(unit):
    if unit == 'G':              #for Giga
        return 1000000000
    if unit == 'M':              #for Mega
        return 1000000 
    if unit == 'k':              #for Kilo
        return 1000
    return 1

def to_numeric(elem): #converting the strings into numbers
    m = sizes_to_num_re.search(elem)
    unit = m.group('unit')
    mult = unit_to_mult(unit)
    amount = float(m.group('amount'))
    return int(amount * mult)

In [84]:
gps['ExtSize'] = gps['Size'].apply(to_numeric) #applying the function 
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000


Ora è possibile riconvertire 'Varies with device' alla sua forma originale, non perdendone così il significato

In [85]:
gps['Size'] = [re.sub('99999999', 'Varies with device', size) for size in gps['Size']] #reconverting the value '99999999' into 'Varies with device'
gps['ExtSize'] = gps['ExtSize'].apply(str)
gps['ExtSize'] = [re.sub('99999999', 'Varies with device', size) for size in gps['ExtSize']]
gps[['App','Size','ExtSize']].head(40)

,App,Size,ExtSize
0,Photo Editor & Candy Camera & Grid & ScrapBook,19M,19000000
1,Coloring book moana,14M,14000000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",8.7M,8700000
3,Sketch - Draw & Paint,25M,25000000
4,Pixel Draw - Number Art Coloring Book,2.8M,2800000
5,Paper flowers instructions,5.6M,5600000
6,Smoke Effect Photo Maker - Smoke Editor,19M,19000000
7,Infinite Painter,29M,29000000
8,Garden Coloring Book,33M,33000000
9,Kids Paint Free - Drawing Fun,3.1M,3100000


##2. Convert the number of installs to a number 

In [86]:
gps.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize            object
dtype: object

In [87]:
gps.groupby('Installs')['Installs'].size() #looking for some "strange" values

Installs
0                    1
0+                  14
1+                  67
1,000+             907
1,000,000+        1579
1,000,000,000+      58
10+                386
10,000+           1054
10,000,000+       1252
100+               719
100,000+          1169
100,000,000+       409
5+                  82
5,000+             477
5,000,000+         752
50+                205
50,000+            479
50,000,000+        289
500+               330
500,000+           539
500,000,000+        72
Name: Installs, dtype: int64

Sotto si può vedere l'unico record che ha un valore non standard per quanto riguarda 'Installs', ovvero senza più; ma dato che nella ricerca del numero il carattere + verrà ignorato, questo non rappresenta un problema

In [88]:
gps[gps.Installs=='0']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize
9148,Command & Conquer: Rivals,FAMILY,NaN,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device,Varies with device


In [89]:
install_to_num = re.compile('(?P<number>[\d,]+)') #the format is num,num,num,...,num as we have seen above

#We find using regex the number in the format 
def installToNumber(installs):
    found = install_to_num.search(installs)
    if(found):
        replacedComma = found.group('number').replace(',','') #replace the commas with nothing
        return int(replacedComma)
    else:
        return np.nan

In [90]:
gps['InstallNumber']=gps['Installs'].apply(installToNumber)

In [91]:
gps[['App','Installs','InstallNumber']].head()

,App,Installs,InstallNumber
0,Photo Editor & Candy Camera & Grid & ScrapBook,"10,000+",10000
1,Coloring book moana,"500,000+",500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...","5,000,000+",5000000
3,Sketch - Draw & Paint,"50,000,000+",50000000
4,Pixel Draw - Number Art Coloring Book,"100,000+",100000


##3. Transform “Varies with device” into a missing value

In [92]:
gps.replace('Varies with device', np.nan,inplace=True)
gps[['App','Size','ExtSize']].head(40)

,App,Size,ExtSize
0,Photo Editor & Candy Camera & Grid & ScrapBook,19M,19000000
1,Coloring book moana,14M,14000000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",8.7M,8700000
3,Sketch - Draw & Paint,25M,25000000
4,Pixel Draw - Number Art Coloring Book,2.8M,2800000
5,Paper flowers instructions,5.6M,5600000
6,Smoke Effect Photo Maker - Smoke Editor,19M,19000000
7,Infinite Painter,29M,29000000
8,Garden Coloring Book,33M,33000000
9,Kids Paint Free - Drawing Fun,3.1M,3100000


##4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [93]:
gps[['App','Current Ver', 'Android Ver']].head()

,App,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,1.0.0,4.0.3 and up
1,Coloring book moana,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,NaN,4.2 and up
4,Pixel Draw - Number Art Coloring Book,1.1,4.4 and up


In [94]:
gps['Android Ver'].value_counts()

4.1 and up       2451
4.0.3 and up     1501
4.0 and up       1375
4.4 and up        980
2.3 and up        652
5.0 and up        601
4.2 and up        394
2.3.3 and up      281
2.2 and up        244
4.3 and up        243
3.0 and up        241
2.1 and up        134
1.6 and up        116
6.0 and up         60
7.0 and up         42
3.2 and up         36
2.0 and up         32
5.1 and up         24
1.5 and up         20
4.4W and up        12
3.1 and up         10
2.0.1 and up        7
8.0 and up          6
7.1 and up          3
4.0.3 - 7.1.1       2
1.0 and up          2
5.0 - 8.0           2
2.2 - 7.1.1         1
7.0 - 7.1.1         1
5.0 - 7.1.1         1
4.1 - 7.1.1         1
5.0 - 6.0           1
Name: Android Ver, dtype: int64

In [95]:
gps['Current Ver'].value_counts()

1.0                                     809
1.1                                     264
1.2                                     178
2.0                                     151
1.3                                     145
1.0.0                                   136
1.0.1                                   119
1.4                                      88
1.5                                      81
1.0.2                                    80
1.6                                      65
1.0.3                                    62
2.1                                      61
3.0                                      59
1.0.4                                    58
1.7                                      53
2.0.0                                    51
1.0.5                                    50
1.1.0                                    49
1.1.1                                    48
1.0.6                                    47
1.2.1                                    46
1.2.0                           

Dopo aver osservato i valori che assumono 'Current Ver' e 'Android Ver', per entrambi procedo a rimuovere tutti i caratteri che non sono numeri

In [96]:
#remove everything that is not a number from Current Ver
gps['CurrVer'] = gps['Current Ver']
gps['CurrVer'].replace('[a-z]+|[A-z]+','', regex = True, inplace = True)
#remove everything that is not a number from Android Ver
gps['AndroVer'] = gps['Android Ver']
gps['AndroVer'].replace('[a-z]+|[A-z]+','', regex = True, inplace = True)
#rimove eventual '.' at the end of the version
gps['CurrVer'].replace(r'(\.+$)','', regex = True, inplace = True )
gps['AndroVer'].replace(r'(\.+$)','', regex = True, inplace = True )

 

In [97]:
gps[['App','Android Ver','Current Ver','AndroVer','CurrVer']].head()

,App,Android Ver,Current Ver,AndroVer,CurrVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.0.3 and up,1.0.0,4.0.3,1.0.0
1,Coloring book moana,4.0.3 and up,2.0.0,4.0.3,2.0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.0.3 and up,1.2.4,4.0.3,1.2.4
3,Sketch - Draw & Paint,4.2 and up,NaN,4.2,NaN
4,Pixel Draw - Number Art Coloring Book,4.4 and up,1.1,4.4,1.1


##5. Remove the duplicates

Per quanto riguarda la gestione dei duplicati ho scelto di utilizzare le Reviews come discriminante per decidere quale fosse il duplicato più recente; i duplicati, infatti, sono dovuti al fatto che la raccolta dati è stata eseguita in periodi temporalmente differenti

In [98]:
len(gps)

10840

Innanzitutto procedo ad eliminare i duplicati che risultano essere esattamente identici

In [99]:
gps.drop_duplicates(inplace = True)
len(gps)

10357

Ora, come detto sopra, utilizzerò Reviews come discriminante per risolvere la questione dei duplicati: in particolare ordinerò il dataset in modo decrescente in base a questa variabile e imporrò l'eliminazione dei duplicati preservando quelli con il numero di Reviews maggiore (dunque i primi)

In [100]:
gps["Reviews"] = pd.to_numeric(gps["Reviews"]) #using Reviews to understand the most recent duplicate
gps = gps.sort_values('Reviews', ascending = False) #sorting in descending order 
gps[gps['App'] == 'Facebook'][['App','Reviews']] #using Facebook as an example

,App,Reviews
2544,Facebook,78158306
3943,Facebook,78128208


In [101]:
gps.drop_duplicates(subset = 'App', keep = 'first', inplace = True)  #keeping only the duplicate with the highest number of Reviews
gps[gps['App'] == 'Facebook'][['App','Reviews']] #did it, as we can see below

,App,Reviews
2544,Facebook,78158306


In [102]:
len(gps)

9659

##6. For each category, compute the number of apps

In questo caso è sufficiente fare un groupby per Category e utilizzare la funzione size(); ho scelto di ordinare le Category da quella con il maggior numero di app a quella con il numero minore

In [103]:
gps.groupby('Category').size().sort_values(ascending = False)

Category
FAMILY                 1878
GAME                    945
TOOLS                   829
BUSINESS                420
MEDICAL                 395
PERSONALIZATION         376
PRODUCTIVITY            374
LIFESTYLE               369
FINANCE                 345
SPORTS                  325
COMMUNICATION           315
HEALTH_AND_FITNESS      288
PHOTOGRAPHY             281
NEWS_AND_MAGAZINES      254
SOCIAL                  239
BOOKS_AND_REFERENCE     222
TRAVEL_AND_LOCAL        219
SHOPPING                202
DATING                  170
VIDEO_PLAYERS           164
MAPS_AND_NAVIGATION     131
FOOD_AND_DRINK          112
EDUCATION               105
ENTERTAINMENT            86
AUTO_AND_VEHICLES        85
LIBRARIES_AND_DEMO       84
WEATHER                  79
HOUSE_AND_HOME           73
EVENTS                   64
ART_AND_DESIGN           61
PARENTING                60
COMICS                   56
BEAUTY                   53
dtype: int64

##7. For each category, compute the average rating 

In questo caso ho fatto un groupby sempre utilizzando Category, ho calcolato il valore medio (che ho denominato Rating) per Category e ho ordinato le categorie per valori di Rating decrescente

In [104]:
gps.groupby('Category')[['Rating']].mean().sort_values(by = 'Rating', ascending = False)

,Rating
Category,
EVENTS,4.435556
ART_AND_DESIGN,4.359322
EDUCATION,4.349038
BOOKS_AND_REFERENCE,4.344970
PERSONALIZATION,4.332215
PARENTING,4.300000
BEAUTY,4.278571
SOCIAL,4.247291
GAME,4.244432


##8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

Per quanto concerne la creazione del dataframe contenente i generi ho deciso di dividere, utilizzando come discriminante ';', ogniqualvolta le tuple contenessero array di generi

In [105]:
gps['GenresList'] = [genre.split(';') for genre in gps['Genres']] #splitting gender's array
genres = gps.GenresList.apply(pd.Series) #genres dataframe
genres = genres.rename(columns = {0:'First_genre', 1:'Second_genre'})
genres.head(30) #so I obtain gender's dataframe

,First_genre,Second_genre
2544,Social,NaN
336,Communication,NaN
2604,Social,NaN
382,Communication,NaN
1879,Strategy,NaN
4005,Tools,NaN
1872,Arcade,NaN
3665,Video Players & Editors,NaN
7536,Tools,NaN
1878,Strategy,NaN


Ora che è stato creato il dataframe dei generi e una volta che è stato creato un dataframe contenente i nomi delle app, basterà unirli sfruttando i rispettivi indici ottenendo così per ogni app tante ripetizioni quanti fossero i generi a loro assegnati

In [106]:
app = gps['App'] #bridgind table
genres_1 = pd.merge(genres, app.to_frame(), left_index = True, right_index = True)\
           .melt(id_vars = ['App'], value_name = "Single_genre")\
           .drop("variable", axis = 1)\
           .dropna() #this is useful to avoid that apps that don't have a second genre appears again
genres_1.head(20) 

,App,Single_genre
0,Facebook,Social
1,WhatsApp Messenger,Communication
2,Instagram,Social
3,Messenger – Text and Video Chat for Free,Communication
4,Clash of Clans,Strategy
5,Clean Master- Space Cleaner & Antivirus,Tools
6,Subway Surfers,Arcade
7,YouTube,Video Players & Editors
8,"Security Master - Antivirus, VPN, AppLock, Boo...",Tools
9,Clash Royale,Strategy


Effettuo un controllo sfruttando l'app citata nella consegna, ovvero Pixel Draw - Number Art Coloring Book; come si può vedere sotto, l'operazione è andata a buon fine 

In [107]:
genres_1[genres_1['App'] == 'Pixel Draw - Number Art Coloring Book']

,App,Single_genre
4831,Pixel Draw - Number Art Coloring Book,Art & Design
14490,Pixel Draw - Number Art Coloring Book,Creativity


##9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

Per svolgere tale compito si è reso necessario esplicitare una funzione ad hoc.
Come riferimento per i generi è stata utilizzata la colonna GenresList creata nel punto precedente; i generi sono stati presi singolarmente in modo da preservare la distinzione tra i generi esplicitata nel punto precedente.

In [108]:
def create_dummies( df, colname ):
    col_dummies = pd.get_dummies(df[colname].apply(pd.Series).stack()).sum(level=0)
    col_dummies.drop(col_dummies.columns[0], axis=1, inplace=True)
    df = pd.concat([df, col_dummies.astype(bool)], axis=1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [109]:
genres_2 = create_dummies(gps, 'GenresList')
genres_2 

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
2544,Facebook,SOCIAL,4.1,78158306,NaN,"1,000,000,000+",Free,0,Teen,Social,...,False,True,False,False,False,False,False,False,False,False
336,WhatsApp Messenger,COMMUNICATION,4.4,69119316,NaN,"1,000,000,000+",Free,0,Everyone,Communication,...,False,False,False,False,False,False,False,False,False,False
2604,Instagram,SOCIAL,4.5,66577446,NaN,"1,000,000,000+",Free,0,Teen,Social,...,False,True,False,False,False,False,False,False,False,False
382,Messenger – Text and Video Chat for Free,COMMUNICATION,4.0,56646578,NaN,"1,000,000,000+",Free,0,Everyone,Communication,...,False,False,False,False,False,False,False,False,False,False
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,...,False,False,False,True,False,False,False,False,False,False
4005,Clean Master- Space Cleaner & Antivirus,TOOLS,4.7,42916526,NaN,"500,000,000+",Free,0,Everyone,Tools,...,False,False,False,False,True,False,False,False,False,False
1872,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,...,False,False,False,False,False,False,False,False,False,False
3665,YouTube,VIDEO_PLAYERS,4.3,25655305,NaN,"1,000,000,000+",Free,0,Teen,Video Players & Editors,...,False,False,False,False,False,False,False,True,False,False
7536,"Security Master - Antivirus, VPN, AppLock, Boo...",TOOLS,4.7,24900999,NaN,"500,000,000+",Free,0,Everyone,Tools,...,False,False,False,False,True,False,False,False,False,False
1878,Clash Royale,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,...,False,False,False,True,False,False,False,False,False,False


##10. For each genre, compute the average rating. What is the genre with highest average?

Come per il caso precedente ho sfruttato il dataframe con i generi distinti e l'ho unito con il dataframe originale e in questo modo ho potuto calcolare il rating medio per ogni singolo genere

In [110]:
genres_3 = genres_1.merge(gps, on = 'App')
genres_3[['App','Genres','Single_genre']].head(30)

,App,Genres,Single_genre
0,Facebook,Social,Social
1,WhatsApp Messenger,Communication,Communication
2,Instagram,Social,Social
3,Messenger – Text and Video Chat for Free,Communication,Communication
4,Clash of Clans,Strategy,Strategy
5,Clean Master- Space Cleaner & Antivirus,Tools,Tools
6,Subway Surfers,Arcade,Arcade
7,YouTube,Video Players & Editors,Video Players & Editors
8,"Security Master - Antivirus, VPN, AppLock, Boo...",Tools,Tools
9,Clash Royale,Strategy,Strategy


In [111]:
genres_3.groupby('Single_genre')[['Rating']].mean().sort_values(by = 'Rating', ascending = False)

,Rating
Single_genre,
Events,4.435556
Puzzle,4.370732
Brain Games,4.358065
Art & Design,4.350000
Books & Reference,4.343275
Personalization,4.332215
Word,4.318182
Creativity,4.306250
Music & Audio,4.300000


Come si può vedere il genere con il rating medio più elevato è 'Events'

In [112]:
genres_3.groupby('Single_genre')['Rating'].mean().idxmax()


'Events'

In [113]:
genres_3.groupby('Single_genre')['Rating'].mean().max()

4.435555555555556

##11. For each app, compute the approximate income, obtain as a product of number of installs and price.

Per svolgere questo compito ho prima di tutto eliminato i caratteri di Price, l'ho convertito in un attributo numerico e infine ho calcolato il guadagno approssimato (calcolato come prodotto tra prezzo e numero di installazioni, come suggerito dalla consegna)

In [121]:
gps['NumPrice'] = [re.sub('\$','', price) for price in gps['Price']]
gps['NumPrice'] = gps['NumPrice'].apply(float)
gps['Income'] = gps['NumPrice'] * gps['InstallNumber']
gps[['App', 'NumPrice', 'InstallNumber', 'Income']].sort_values(by = 'Income', ascending = False).head(20)

,App,NumPrice,InstallNumber,Income
2241,Minecraft,6.99,10000000,69900000.0
5351,I am rich,399.99,100000,39999000.0
5356,I Am Rich Premium,399.99,50000,19999500.0
4034,Hitman Sniper,0.99,10000000,9900000.0
7417,Grand Theft Auto: San Andreas,6.99,1000000,6990000.0
2883,Facetune - For Free,5.99,1000000,5990000.0
5578,Sleep as Android Unlock,5.99,1000000,5990000.0
8804,DraStic DS Emulator,4.99,1000000,4990000.0
4367,I'm Rich - Trump Edition,400.00,10000,4000000.0
5354,I am Rich Plus,399.99,10000,3999900.0


##12. For each app, compute its minimum and maximum Sentiment_polarity

Per ogni app ho individuato la massima e la minima Sentiment_polarity per poi inserirle entrambe in un unico dataframe

In [126]:
gps_ur.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [125]:
#maximum
data_max = gps_ur.groupby('App')['Sentiment_Polarity'].max()
data_max = pd.DataFrame(data_max)
data_max = data_max.rename(columns = {'Sentiment_Polarity':'Max_Sentiment_Polarity'})
data_max.head(10)

,Max_Sentiment_Polarity
App,
10 Best Foods for You,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000
11st,1.000000
1800 Contacts - Lens Store,0.838542
1LINE – One Line with One Touch,1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000
21-Day Meditation Experience,0.587500
"2Date Dating App, Love and matching",1.000000
2GIS: directory & navigator,1.000000


In [130]:
#minimum
data_min = gps_ur.groupby('App')['Sentiment_Polarity'].min()
data_min = pd.DataFrame(data_min)
data_min = data_min.rename(columns = {'Sentiment_Polarity':'Min_Sentiment_Polarity'})
data_min.head(10)

,Min_Sentiment_Polarity
App,
10 Best Foods for You,-0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.112500
11st,-1.000000
1800 Contacts - Lens Store,-0.300000
1LINE – One Line with One Touch,-0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,-0.800000
21-Day Meditation Experience,-0.265625
"2Date Dating App, Love and matching",-0.645833
2GIS: directory & navigator,-0.375000


Ora procedo a fare il merge dei dataframe sopra

In [129]:
data_sentiment_polarity = data_max.merge(data_min, left_index = True, right_index = True)
data_sentiment_polarity.head(10)

,Max_Sentiment_Polarity,Min_Sentiment_Polarity
App,,
10 Best Foods for You,1.000000,-0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.112500
11st,1.000000,-1.000000
1800 Contacts - Lens Store,0.838542,-0.300000
1LINE – One Line with One Touch,1.000000,-0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000,-0.800000
21-Day Meditation Experience,0.587500,-0.265625
"2Date Dating App, Love and matching",1.000000,-0.645833
2GIS: directory & navigator,1.000000,-0.375000
